In [1]:
#Oppgave 2
import requests
import pandas as pd
import json
from pandasql import sqldf

# Henter inn dataen
API_URL = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
headers = {"User-Agent": "MyWeatherApp/1.0 (contact@example.com)", "Accept": "application/json"}
params = {"lat": 59.91, "lon": 10.75}  # Oslo koordinater

response = requests.get(API_URL, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    with open("../data/weather_data.json", "w") as f:
        json.dump(data, f, indent=4)
    print("Data hentet og lagret som JSON")
else:
    print("Feil ved henting av data! Statuskode:", response.status_code)
    exit()

#Behandler data med Pandas
timeseries = data["properties"]["timeseries"]
rows = []

for entry in timeseries:
    time = entry["time"]
    details = entry["data"]["instant"]["details"]
    rows.append({"Tidspunkt": time, **details})

df = pd.DataFrame(rows)

#Oversetter til norsk
column_mapping = {
    "air_pressure_at_sea_level": "Lufttrykk ved havnivå",
    "air_temperature": "Lufttemperatur",
    "wind_speed": "Vindhastighet",
    "wind_from_direction": "Vindretning",
    "relative_humidity": "Relativ fuktighet",
    "cloud_area_fraction": "Skydekke"
}
df.rename(columns=column_mapping, inplace=True)

df.to_csv("../data/weather_data.csv", index=False)
print("Data lagret som CSV")

#Vurdering av kilde
source_info = {
    "Kilde": "MET Norway",
    "Pålitelighet": "Høy",
    "Tilgjengelighet": "Åpen tilgang, krever User-Agent header",
    "Format": "JSON",
    "Oppdateringsfrekvens": "Løpende oppdateringer",
}
print("Vurdering av kilde:")
for key, value in source_info.items():
    print(f"{key}: {value}")

#Data analyse med PandaSQL
pysqldf = lambda q: sqldf(q, globals())
query = """
SELECT Tidspunkt, Lufttemperatur, Vindhastighet
FROM df
WHERE Lufttemperatur IS NOT NULL
ORDER BY Tidspunkt ASC
LIMIT 5;
"""
result = pysqldf(query)
print("\nEksempel på analyserte data:")
print(result)


Data hentet og lagret som JSON
Data lagret som CSV
Vurdering av kilde:
Kilde: MET Norway
Pålitelighet: Høy
Tilgjengelighet: Åpen tilgang, krever User-Agent header
Format: JSON
Oppdateringsfrekvens: Løpende oppdateringer

Eksempel på analyserte data:
              Tidspunkt  Lufttemperatur  Vindhastighet
0  2025-03-30T12:00:00Z             7.3            3.9
1  2025-03-30T13:00:00Z             8.0            4.0
2  2025-03-30T14:00:00Z             8.4            3.9
3  2025-03-30T15:00:00Z             8.8            3.5
4  2025-03-30T16:00:00Z             8.7            3.2


In [2]:
#Oppgave 3
#Håndtering av manglende verdier 
def clean_data(df):
    df.dropna(inplace=True)  
    return df

df_cleaned = clean_data(df)
print("\nData etter håndtering av manglende verdier:")
print(df_cleaned.head())

#List comprenhensions 
df_cleaned["Tidspunkt"] = [t.replace("T", " ") for t in df_cleaned["Tidspunkt"]]
print("\nTidspunkt formatert:")
print(df_cleaned["Tidspunkt"].head())

#Manipulerer data med Pandas SQL 
query = """
SELECT Tidspunkt, Lufttemperatur, Vindhastighet
FROM df_cleaned
WHERE Lufttemperatur > 0
ORDER BY Vindhastighet DESC
LIMIT 5;
"""
result = pysqldf(query)
print("\nEksempel på filtrerte data:")
print(result)

#Forventede uregelmessigheter og håndtering
print("\nForventede uregelmessigheter og håndtering:")
print("- Manglende verdier: Løst med dropna().")
print("- Uregelmessige tidspunkter: Formatert med list comprehension.")
print("- Ekstreme verdier: Kan analyseres med ytterligere filtrering.")



Data etter håndtering av manglende verdier:
              Tidspunkt  Lufttrykk ved havnivå  Lufttemperatur  Skydekke  \
0  2025-03-30T12:00:00Z                 1005.2             7.3      95.2   
1  2025-03-30T13:00:00Z                 1006.0             8.0      96.4   
2  2025-03-30T14:00:00Z                 1007.0             8.4      95.9   
3  2025-03-30T15:00:00Z                 1007.8             8.8      82.3   
4  2025-03-30T16:00:00Z                 1008.9             8.7      47.1   

   Relativ fuktighet  Vindretning  Vindhastighet  
0               87.3         39.3            3.9  
1               84.3         28.8            4.0  
2               81.1         22.4            3.9  
3               79.2         24.1            3.5  
4               78.5         36.0            3.2  

Tidspunkt formatert:
0    2025-03-30 12:00:00Z
1    2025-03-30 13:00:00Z
2    2025-03-30 14:00:00Z
3    2025-03-30 15:00:00Z
4    2025-03-30 16:00:00Z
Name: Tidspunkt, dtype: object

Eksempel